## McElroy-only podcasts
Chose all of their podcasts because of the consistent formatting
Analyze language change of each host over time

In [1]:
%store -r mcelroy

In [2]:
import re
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
mcelroy.index.value_counts()

mbmbam                32
sawbones              30
wonderful             29
shmanners             28
the adventure zone    19
Name: Podcast, dtype: int64

In [4]:
mcelroy.columns

Index(['Episode', 'Title', 'Year', 'Text'], dtype='object')

In [5]:
mbmbam_sample = mcelroy.loc['mbmbam'].Text[5]
mbmbam_sample[-1000:]

"\n \nGriffin:  It'll be a hoot. Uh, so, y'all want the final? \n \nJustin:  Oh yeah. \n \nTravis:  Please, Griffin. \n \nGriffin:  Final Yahoo. This one was sent in by hat trick, Graham Robuck. \nCrushed it. Thank you, Graham. It‘s Yahoo Answers user Josh, who asks, \n―How do I get pickle smell off body? Quick!‖ \n \nTravis:  [laughs]  \n \nGriffin:  I cannot—I'm gonna read the next—―My mom can't know I've been \neating her pickles! She gets off work soon, and she‘s gonna smell them \nunless I get it off! Tried shower, still stinks!‖  \n \nTravis:  [laughing] \n \nGriffin:  ―Update: She knows.‖ \n \nJustin:  [laughs] My name is Justin McElroy. \n \nTravis:  [laughing uncontrollably] I'm Travi—I'm Travis McElroy! \n \nJustin:  That‘s Travis McElroy.    \nGriffin:  I'm Griffin McElroy. \n \nJustin:  This has been My Brother, My Brother, and Me. Kiss your dad \nsquare on the lips. \n \nTravis:  [still laughing] \n \n[theme music plays] \n \nMaximumFun.org \nComedy and culture. \nArtist o

In [6]:
# mbmbam - specific speech parser
def maxfun_parse_speech(text, host_names):    
    
    text = re.sub('\n', '', text)
    host_dict = {}
    
    for host in host_names:
        other_hosts_regex = '|'.join(set([h for h in host_names if h != host]))
        
        host_regex = r'(?<=' + host + ').*?(?=' + other_hosts_regex + ')'
        host_dict[host] = [speech.strip().lower() for speech in re.findall(host_regex, text)]
    
    
    for name, speech in list(host_dict.items()):
        new_name = ''.join([c for c in name if c.isalpha() or c.isspace()])
        new_name = new_name.strip().lower()
        host_dict[new_name] = speech
        del host_dict[name]
    
    return host_dict

In [7]:
mbmbam = mcelroy.loc[mcelroy.index == 'mbmbam']
mbmbam['Host_parsed'] = mbmbam.Text.map(lambda text: maxfun_parse_speech(text, ['Griffin:  ','Travis:  ','Justin:  ']))
mbmbam.Host_parsed[10]['justin'][:10]

<ipython-input-7-949ebf6c3a47>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbmbam['Host_parsed'] = mbmbam.Text.map(lambda text: maxfun_parse_speech(text, ['Griffin:  ','Travis:  ','Justin:  ']))


['hello, everybody, and welcome to my brother, my brother and me, an advice show for the modern era.  i‘m your oldest brother, justin mcelroy!',
 'oh, god.',
 'uh...',
 '[sighs]',
 'now, i thought that was what the law was doing already, pretty much.',
 'yeah.',
 '[laughs] yeah.',
 'do you think it would be—do you think that batman would have the same weight if you were like, ―why won‘t you reveal your secret identity?‖  and he actually had to say with a straight face, ―to protect my butler.‖',
 '―i don‘t want to—[crosstalk].‖',
 '―i have a nice house and a cool butler and i don‘t want either of them to get messed up!‖']

In [8]:
def redacter(text, redacted_words_list):
    
    text = text[500:]
    text = text.lower()
    text = re.sub('\n','', text)
    
    for w in redacted_words_list:
        text = re.sub(w.lower(), 'REDACTED', text)
    
    return text

redacter(mbmbam_sample, ['griffin','travis','justin','brother','griff','trav', 'mcelroy','mbmbam', 'maxfun', 'sexpert'])[:1000]       

" REDACTED, my REDACTED, and me, an advice show for the modern era. i'm your oldest REDACTED, REDACTED REDACTED.  REDACTED:  i‘m your middlest REDACTED, REDACTED REDACTED.  REDACTED:  and i'm your sweet baby REDACTED, REDACTED REDACTED. [sings a silly tune] doot doot doo doo dooooot!  REDACTED:  my my my, is that the piper?  REDACTED:  he‘s back. and i know that—real quick, guys. we should've talked about this before we started recording. i know this is gonna be our super chill sort of REDACTEDdrive that‘s going for four weeks, but i thought we could go, like, this first week, like, pretty fuckin‘ heavy on it.  REDACTED:  it‘s me, the pipahhh!  REDACTED:  yeah, exactly that type of energy. i love it.  REDACTED:  okay.  REDACTED:  i am the pipahhh!   REDACTED:  yeah. and—  REDACTED:  i—the bill has come due, respectfully!    REDACTED:  it‘s unbelievable to me that you would not know this at this point, but we are a part of the maximum fun network, which is artist-owned. that‘s right, we

In [9]:
mbmbam_redacted_words = ['griffin','travis','justin','brother','griff','trav', 'mcelroy','mbmbam', 'maxfun', 'bob ball']
mbmbam['Redacted_text'] = mbmbam.Text.map(lambda t: redacter(t, mbmbam_redacted_words))
mbmbam.sample(3)

<ipython-input-9-770e9490addd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mbmbam['Redacted_text'] = mbmbam.Text.map(lambda t: redacter(t, mbmbam_redacted_words))


,Episode,Title,Year,Text,Host_parsed,Redacted_text
Podcast,,,,,,
mbmbam,535,ethical joker termination,2020,MBMBaM 535: Ethical Joker Termination \nPublis...,"{'griffin': ['oh.', 'and this is griffin mcelr...","d welcome to my REDACTED, my REDACTED and me, ..."
mbmbam,536,ratashootie,2020,MBMBaM 536: Ratashootie \nPublished on Novembe...,"{'griffin': ['[singing a single note]', 'ba-bu...","REDACTED: hello, everybody, and welcome to m..."
mbmbam,534,the hang in there burger,2020,MBMBaM 534: The Hang-in-There Burger \nPublish...,"{'griffin': ['good evening.', 'please be seate...","ters, plays] REDACTED: hello, everybody, and..."


In [10]:
sawbones_sample = mcelroy.loc['sawbones'].Text[5]
sawbones_sample[-1000:]

'ntro and outro of our program. You know, that’s a good use of— \nthat’s a good Max Fun Drive point. We got that song “Medicines” from a \nfree music library that was posted with Creative Commons, you could \nmake the show and, you know, you can use the song however you like. \nBut as the show became more popular and we started to make money \nfrom the show, we were able to pay The Taxpayers. I mean they didn’t \nask for anything, but we were able to pay The Taxpayers and just say, \nlike, “Hey, this show is making money now thanks to support from our \ndonors, like, we would like to pass some of that onto you,” and that’s all \nthanks to, you know, the people— \nSydnee:  Thanks to you! \nJustin:  Thanks to you, specifically. That is gonna do it for us for this \nweek, so until next time, my name is Justin McElroy. \nSydnee:  I’m Sydnee McElroy. \nJustin:  And as always, don’t drill a hole in your head.  \n[theme music plays] \nMaximumFun.org \nComedy and culture. \nArtist owned.   Aud

In [11]:
sawbones = mcelroy.loc[mcelroy.index == 'sawbones']
sawbones['Host_parsed'] = sawbones.Text.map(lambda text: maxfun_parse_speech(text, ['Sydnee:  ','Justin:  ']))
sawbones.Host_parsed[10]['sydnee'][:10]

<ipython-input-11-278e88a9e3bb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sawbones['Host_parsed'] = sawbones.Text.map(lambda text: maxfun_parse_speech(text, ['Sydnee:  ','Justin:  ']))


["and i'm sydnee mcelroy.",
 'no. this is an issue that has existed for a very, very long time, and i do not believe that i have the ability, or power, or even all of the knowledge to solve it right now. but i do wanna share it with everyone.',
 "i'll do my best.",
 '—[crosstalk] that you\'re hungry. i, uh—we\'re a little late this week on our episode because i am on hospital service, and that makes me a little busier than usual. i was all ready, it was just finding the time to record.  but we, uh—before i left work today i said, "okay, i gotta go record a podcast, i\'ll see you guys later." and my residents, some of \'em listened and they were like, "hey, what are you doin\' your show on?"    and... i couldn\'t—i didn\'t know how to explain at work what i was gonna do our show on. uh, because of—the inspiration is not exactly a music video that i could... is—is not appropriate for the workplace environment, let\'s say.',
 "uh... i guess. i guess that's true. [laughs quietly] but gener

In [12]:
sawbones_redacted_words = ['sydnee','justin','history','medicine','medical','mcelroy','sawbones', 'maxfun', 'clint', 'marital','misguided']
sawbones['Redacted_text'] = sawbones.Text.map(lambda t: redacter(t, sawbones_redacted_words))
sawbones.sample(3)

<ipython-input-12-ce5e60af855f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sawbones['Redacted_text'] = sawbones.Text.map(lambda t: redacter(t, sawbones_redacted_words))


,Episode,Title,Year,Text,Host_parsed,Redacted_text
Podcast,,,,,,
sawbones,342,our 202,2020,Sawbones 342: Our 2020 Flu Shot Extravaganza \...,"{'sydnee': ['and i‟m sydnee mcelroy.', 'oh, th...","stin: hello everybody, and welcome to REDACTE..."
sawbones,356,mewing,2021,"Sawbones 356: Mewing \nPublished February 9th,...",{'sydnee': ['and i'm sydnee mcelroy. i got rea...,nd welcome to [sing-song] REDACTED: a REDACTED...
sawbones,332,operation warp speed,2020,Sawbones 332: Operation Warp Speed \nPublished...,"{'sydnee': ['and i’m sydnee mcelroy.', 'that’s...","everybody, and welcome to REDACTED: a REDACTED..."


In [13]:
wonderful_sample = mcelroy.loc['wonderful'].Text[5]
wonderful_sample[-1000:]

"ge people to support it because they let you pick the shows that you \nwanna support. \n \nGriffin:  Yeah. \n \nRachel:  You know? And all of these artists, um, really appreciate your support, \nand also a lot of them live off of the support. \n \nGriffin:  Of course, yeah. \n \nRachel:  So, uh, anything that you're able to invest is appreciated. \n \nGriffin:  Yeah, and you'll get cool stuff. Um, so, yeah. That's gonna do it for us. \nWe will be back next week, and the next episode, y'all? [fake laughter] No \nspoilers, but it's gonna be a real barn burner. \n \nRachel:  Explosions! Suspense! Chases! \n \n[pauses] \n \nGriffin:  Super, uh— \n \nRachel:  Romance?    \nGriffin:  —romance? [pauses] Uh, b—battle... for the ages. \n \nRachel:  [holding back laughter] Deceit. \n \nGriffin:  S—villainy. \n \nRachel:  [laughs]  \n \nGriffin:  Kissing? \n \nRachel:  Ooh! \n \nGriffin:  [high pitched] Oooh! \n \nRachel:  [laughs]  \n \n[theme music plays] \n \n \nMaximumfun.org. \nComedy and C

In [14]:
wonderful = mcelroy.loc[mcelroy.index == 'wonderful']
wonderful['Host_parsed'] = wonderful.Text.map(lambda text: maxfun_parse_speech(text, ['Rachel:  ','Griffin:  ']))
wonderful.Host_parsed[10]['rachel'][:10]

<ipython-input-14-f91e5697fbb2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wonderful['Host_parsed'] = wonderful.Text.map(lambda text: maxfun_parse_speech(text, ['Rachel:  ','Griffin:  ']))


['hello, this is rachel mcelroy.',
 'and this is wonderful!',
 '[pause] you really dedicated your life to being a beard man now?',
 'mm-hmm.',
 'mm-hmm.',
 'oh. great.',
 'love it…',
 '[laughs]',
 "yeah, y'know what? i do, actually.",
 'i was thinking about it, and i will say, uh… adding a recipe to the rotation.']

In [15]:
wonderful_redacted_words = ['rachel','griffin','wonderful', 'maxfun', 'mcelroy']
wonderful['Redacted_text'] = wonderful.Text.map(lambda t: redacter(t, wonderful_redacted_words))
wonderful.sample(3)

<ipython-input-15-a74b4fd65061>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wonderful['Redacted_text'] = wonderful.Text.map(lambda t: redacter(t, wonderful_redacted_words))


,Episode,Title,Year,Text,Host_parsed,Redacted_text
Podcast,,,,,,
wonderful,140,the atomic collage,2020,Wonderful! 140: The Atomic Collage \nPublished...,"{'rachel': ['hello, this is rachel mcelroy.', ...",s always brought a lot of joy and excitement t...
wonderful,161,ricolaaaaaahhhh,2020,Wonderful! 161: Ricolaaaaaahhhh \nPublished De...,"{'rachel': ['hi, this is rachel mcelroy.', 'an...","feel the best. but i still love her, despite ..."
wonderful,144,inside out larping,2020,\n ...,"{'rachel': ['hello, this is rachel mcelroy.', ...","u‟re talking about crane pose. REDACTED: oh,..."


In [16]:
shmanners_sample = mcelroy.loc['shmanners'].Text[5]
shmanners_sample[-1000:]
shmanners = mcelroy.loc[mcelroy.index == 'shmanners']
shmanners['Host_parsed'] = shmanners.Text.map(lambda text: maxfun_parse_speech(text, ['Teresa:  ','Travis:  ']))
shmanners.Host_parsed[10]['teresa'][:10]
shmanners_redacted_words = ['travis','teresa','shmanners', 'maxfun', 'mcelroy']
shmanners['Redacted_text'] = shmanners.Text.map(lambda t: redacter(t, shmanners_redacted_words))
shmanners.sample(3)

" you to everybody who has supported Shmanners in the past \nand made us able to pay Alex to help us out! Because y'know, I don't know \nif folks know this – we've got two kids. \n \nTeresa:  And you should pay people for the work they do for you. \n \nTravis:  Yes, that too. But I won't pay our kids for— \n \nTeresa:  No, they don‘t do any work for us. \n \nTravis:  Oh, we‘re talking about Al—okay, great. \n \nTeresa:  I have just managed to teach Bebe how to fold a washcloth. She \nhas a lot of work to do. \n \nTravis:  That‘s fair.    \nTeresa:  [laughs] \n \nTravis:  Alright, and that‘s gonna do it for us jo—mm? ‗That‘s gonna do it \nfor us jokes‘ is what I started to say? Ugh. \n \nTeresa:  [laughs] \n \nTravis:  That‘s gonna do it for us, folks. Join us again next time. \n \nTeresa:  No RSVP required. \n \nTravis:  You've been listening to Shmanners. \n \nTeresa:  Manners, Shmanners! Get it! \n \n[theme music plays] \n \nMaximumFun.org \nComedy and culture. \nArtist owned.  \nAud

<ipython-input-16-9d498e443360>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shmanners['Host_parsed'] = shmanners.Text.map(lambda text: maxfun_parse_speech(text, ['Teresa:  ','Travis:  ']))


['agatha christie done it!',
 "and i'm your wife host, teresa mcelroy.",
 "it's extraordinary etiquette...",
 'hello, dear.',
 '[laughs] uh—',
 '—not quite that close, [holding back laughter] but we are a lot closer than we usually are?',
 '[laughs]',
 'you are touching me.',
 'stop it.',
 '[laughs]']

<ipython-input-16-9d498e443360>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shmanners['Redacted_text'] = shmanners.Text.map(lambda t: redacter(t, shmanners_redacted_words))


,Episode,Title,Year,Text,Host_parsed,Redacted_text
Podcast,,,,,,
shmanners,233,costumes,2020,Shmanners 233: Costumes \nPublished October 23...,"{'teresa': ['i don't know, what?', '[laughs] i...","for ordinary occasions. hello, my dove. REDAC..."
shmanners,215,idioms pt1,2020,\n ...,"{'teresa': ['what?', 'and i‘m your wife host, ...",it‘s extraordinary etiquette… REDACTED: … for...
shmanners,242,new years eve traditions,2020,Shmanners 242: New Year’s Eve Traditions \nPub...,{'teresa': ['it's shmanners! [theme music pla...,"vis: how are you? REDACTED: you know, uh, t..."


In [17]:
taz_sample = mcelroy.loc['the adventure zone'].Text[5]
taz_sample[-1000:]
taz = mcelroy.loc[mcelroy.index == 'the adventure zone']
taz['Host_parsed'] = taz.Text.map(lambda text: maxfun_parse_speech(text, ['Griffin: ','Travis: ','Justin: ','Clint: ']))
taz.Host_parsed[10]['travis'][:10]
taz_redacted_words = ['travis','justin','griffin','clint','maximum','fun','adventure','zone','fantasy', 'firbolg', 'argo', 'dad']
taz['Redacted_text'] = taz.Text.map(lambda t: redacter(t, taz_redacted_words))
taz.sample(3)

"nd I just want to say, on a personal level, uh… \ngoing into this, we weren‘t really sure, like, what the response was gonna \nbe, ‗cause we know these are very uncertain times, and a lot of people have \nfound themselves in pretty dire financial positions, not knowing what‘s \ncoming in the future. And the fact that so many of you have still, uh, \ny'know, made the decision to become new and upgrading members… it just \nreally means a lot.  \n \nAnd that‘s not to pressure people who aren't able to, because I know y'all \nare supporting in whatever way you can. If it‘s sharing links, or just like,   sharing the shows or whatever, all of that matters. And just all that support \nmeans a lot to us, so thank you very much. \n \nGriffin:  Uh, Yeah. Let‘s uh… let‘s wrap it up. \n \nJustin:  I think we just did. \n \nGriffin:  Oh, oka—oh. Okay. \n \nTravis:  Okay, bye! \n \n[theme music plays] \n \nMaximumFun.org \nComedy and culture. \nArtist owned.  \nAudience supported. \n \n[Maximum Fun

<ipython-input-17-5e06ad2a2af9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['Host_parsed'] = taz.Text.map(lambda text: maxfun_parse_speech(text, ['Griffin: ','Travis: ','Justin: ','Clint: ']))


['argo and firbolg, you are awoken from your slumber by a pounding on the door. it‘s getting, uh, progressively louder, and let‘s say, frantic.',
 'no.',
 'unless—unless it‘s preceded by dave, i don‘t want any foley, dad.',
 "yeah, we really nailed—well, you just gotta know when you've hit the top, y'know? when you've peaked, you gotta come off of it.  so, there‘s still a pounding on the door, but this might—",
 'it has now progressed to the sound of… let‘s say, a battering ram?',
 'no no no. it wasn‘t a metaphor – it literally is a battering ram.',
 "you open the door, and you see rainer in her chair come flying through, and she‘s been slamming her chair backwards into the door to try to get your attention. so she comes hovering into the room backwards, and kind of like, skids to a halt in the air, and says…  rainer:  oh, hey! uh, you're up! great! um…  argo:  we really didn‘t have much choice with your chair antics there, friend.  rainer:  yeah. yeah, sorry about that! um, uh, kind o

<ipython-input-17-5e06ad2a2af9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taz['Redacted_text'] = taz.Text.map(lambda t: redacter(t, taz_redacted_words))


,Episode,Title,Year,Text,Host_parsed,Redacted_text
Podcast,,,,,,
the adventure zone,163,tthe adventure zone ep z07 maxfundrive 202,2020,The The Adventure Zone Zone: MaxFunDrive 2020 ...,"{'griffin': ['yeah.', 'i hope not that.', 'yea...",we are going to very masturbatorily talk abou...
the adventure zone,170,graduation27 long way down,2020,"The Adventure Zone: Graduation – Ep. 27, Long ...","{'griffin': ['just off to the races. ooh, i go...",", fast forward 49 and a half or so years, and ..."
the adventure zone,176,graduation32 by a haircut,2021,"The Adventure Zone: Graduation – Ep. 32, By a ...","{'griffin': ['whoaaa!', 'that‘s wild.', 'there...","die. their first step? getting a haircut, of ..."


In [18]:
mcelroy = pd.concat([mbmbam, sawbones, wonderful, shmanners, taz]).reset_index()
mcelroy

,Podcast,Episode,Title,Year,Text,Host_parsed,Redacted_text
0,mbmbam,513,mother cruncher,2020,MBMBaM 513: Mother Cruncher \nPublished on Jun...,{'griffin': ['i'm your sweet baby brother and ...,"her, my REDACTED, and me, an advice show for t..."
1,mbmbam,514,kickeo,2020,"MBMBaM 514: Kickeo \nPublished on June 15th, 2...","{'griffin': ['i'm your sweet brother, griffin....","nd me, an advice show for the modern era. i'm ..."
2,mbmbam,515,viscous bod,2020,MBMBaM 515: Viscous Bod \nPublished on June 22...,"{'griffin': ['i'm your sweet baby brother, gri...",", my REDACTED, and me, an advice show for the ..."
3,mbmbam,516,radical gamer facts,2020,MBMBaM 516: Radical Gamer Facts \nPublished ...,{'griffin': ['you say that—you do that a lot t...,", my REDACTED, and me, an advice show for the ..."
4,mbmbam,517,pepperoni prescription,2020,MBMBaM 517: Pepperoni Prescription \nPublished...,{'griffin': ['i don‘t even want to—i don‘t eve...,it‘s always brought a lot of joy and excitemen...
...,...,...,...,...,...,...,...
133,the adventure zone,171,graduation28 business plan,2020,"The Adventure Zone: Graduation – Ep. 28, Busin...",{'griffin': ['i raise my hand. order: yes? ...,"or mortals to comprehend, but… basically, we a..."
134,the adventure zone,173,graduation29 princi pal,2020,"The Adventure Zone: Graduation – Ep. 29, Princ...",{'griffin': ['i thought you were gonna make us...,"‘s really good. REDACTED: uh, 24. REDACTED:..."
135,the adventure zone,174,graduation30 take your firbolg to work day,2020,"The Adventure Zone: Graduation – Ep. 30, Take ...","{'griffin': ['oh nooo!', 'man, he seemed so co...",oned into helping higglemas wiggenstaff cure h...
136,the adventure zone,175,graduation31 high on the hog,2020,"The Adventure Zone: Graduation – Ep. 31, High ...","{'griffin': ['yeah.', 'um, and just to sort of...",ight guild from the inside. their hope is that...


## Analysis

see how the linguistic style changed over time, based on episode number

In [19]:
import spacy
from spacy.lang.en import English

nlp = spacy.load('en_core_web_md')

In [20]:
mcelroy['Tokens'] = mcelroy.Text.map(nlp)
mcelroy.Tokens[0][:100]

MBMBaM 513: Mother Cruncher 
Published on June 8th, 2020 
Listen on TheMcElroy.family 
 
Intro (Bob Ball):  The McElroy brothers are not experts, and their advice 
should never be followed. Travis insists he's a sexpert, but if there's a 
degree on his wall, I haven't seen it. Also, this show isn't for kids, which I 
mention only so the babies out there will know how cool they are for 
listening. What's up, you cool baby?

In [22]:
mcelroy['Redacted_tokens'] = mcelroy.Redacted_text.map(lambda t: nlp(t.lower()))
mcelroy.Redacted_tokens[0][:100]

her, my redacted, and me, an advice show for the modern era. i'm your oldest redacted, redacted redacted.  redacted:  i am your middlest redacted, and i have been and will always be, redacted redacted.  redacted:  i'm your sweet baby redacted and 30 under 30 media luminary, redacted redacted. and folks… we beefed it, didn‘t we?  i will be the first to admit that we beefed it, and…  redacted:  you mean regarding laughter

In [27]:
mcelroy.loc[1, 'Podcast']

'mbmbam'

In [153]:
def extract_host_speech(dataframe):
    
    all_speakers = {}
    
    for i in range(len(dataframe)):
        speaker_dict = dataframe.iloc[i, 5]
        for p, s in speaker_dict.items():
            if p not in all_speakers.keys():
                all_speakers[p] = s
            else: 
                all_speakers[p] += s          
            
    
    return all_speakers
        
all_speakers = extract_host_speech(mcelroy)
    

## Train a tfidf model to predict speaker based on sentence

In [156]:
speakers = []

for i, s in all_speakers.items():
    for t in s:
        if isinstance(t, str):
            speakers.append([i, t])
        else:
            for f in t:
                speakers.append((i, f))

In [157]:
speakers

[['griffin',
  "i'm your sweet baby brother and 30 under 30 media luminary, griffin mcelroy. and folks… we beefed it, didn‘t we?  i will be the first to admit that we beefed it, and…"],
 ['griffin',
  'regarding laughter and love. thank you travis, yes. we have gotten some criticism, i think, over, um… our selection of… the motto. of the tagline. whatever you wanna call it. and… i have actually gathered everybody here today, for this press conference. [makes a whispering sound] hububububub.'],
 ['griffin',
  "to announce… my… i can't—i fuckin‘ can't believe i'm saying this."],
 ['griffin', 'my support for twenty honey: down to beezness.'],
 ['griffin',
  'and this has never happened before in the series of the sho—the run of the show! that we have—'],
 ['griffin',
  "that we have endorsed a switcheroonie in the middle of the year. folks, it‘s fuckin‘ june. we‘re like, halfway through this shit. and i'm—i'm suggesting a switcheroonie."],
 ['griffin',
  "i'm leaning hard into switcheroon

In [158]:
speaker_df = pd.DataFrame(speakers, columns=['Speaker','Speech'])
speaker_df

for t in speaker_df.Speech:
    if isinstance(t, list):
        print(t)

,Speaker,Speech
0,griffin,i'm your sweet baby brother and 30 under 30 me...
1,griffin,"regarding laughter and love. thank you travis,..."
2,griffin,to announce… my… i can't—i fuckin‘ can't belie...
3,griffin,my support for twenty honey: down to beezness.
4,griffin,and this has never happened before in the seri...
...,...,...
803508,clint,[laughs]
803509,clint,"i'm gonna do a—i'm gonna do a, uh, macgyver ki..."
803510,clint,"oh, god, do it in my pocket?"
803511,clint,[imitates an angel singing]


In [166]:
speaker_df.to_csv('mcelroy_df.csv')
# exported to crc for ML